## Fasttext

following tutorial https://fasttext.cc/docs/en/supervised-tutorial.html

## TODO
- prepare an input file with categories and their keywords
- prepare input file with categories and single keywords
- split datasets in training and validation
- run simple model settings (default as above)
- balance datasets for training 
 - feed GOOOGLE search words!

 
- try to use it as a word vectorizer and measure distance?

In [305]:
## load the data
import sys
# Add src folder to the path
sys.path.append('../src/')
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data
# Specify wanted time range
start_date = '2017-01-01'
end_date = '2022-01-01'
# Path to data
data_file = '/home/anya_m/data/dw-project-data/CMS_2010_to_June_2022_ENGLISH.json'
# Load and extract data within time range
df_subset = get_data(data_file, start_date, end_date)
# Cleans keywords and saves data as a dataframe
#make_cleaned_keywords_df(df_subset, start_date, end_date)

Getting data...
Loading data DONE. Number of articles is 175659
Extracting data DONE. Number of articles from 2017-01-01 to 2022-01-01 is 69432


In [326]:
# Load DW data
import pandas as pd
start_date = '2017-01-01' #'2019-01-01' #'2021-01-01'
end_date = '2022-01-01'
filepath = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'
df = pd.read_json(filepath, orient ='split', compression = 'infer')

In [307]:
display(len(df))
len(df_subset)

69432

69432

In [533]:
# Load Google data
df_google = pd.read_json('../data/interim/2021-daily-trending-searches.json', orient ='split', compression = 'infer')

In [534]:
# Keeps only google data within DW data date range
start_date_dw=start_date
end_date_dw=end_date
df_google.sort_values(by ='date', inplace = True) 
mask = (pd.to_datetime(df_google['date']) > start_date_dw) & (pd.to_datetime(df_google['date']) <= end_date_dw)
df_google_subset = df_google.loc[mask].copy()
google_topic=df_google_subset['topic_title']
google_topic_type=df_google_subset['topic_title']

In [535]:
df_google_subset.head()
#interesting columns:
# df_google_subset['topic_title']
# df_google_subset['topic_type']

,value,formattedValue,link,topic_mid,topic_title,topic_type,date,location
0,3900,"+3,900%",/trends/explore?q=/m/01wf86y&date=2021-01-01+2...,/m/01wf86y,Cyndi Lauper,American singer,2021-01-01,US
24,60,+60%,/trends/explore?q=/m/02vx4&date=2021-01-01+202...,/m/02vx4,Football,Sports,2021-01-01,US
23,60,+60%,/trends/explore?q=/m/0frq6&date=2021-01-01+202...,/m/0frq6,Pork,Meat,2021-01-01,US
22,60,+60%,/trends/explore?q=/m/0735l&date=2021-01-01+202...,/m/0735l,Streaming media,Topic,2021-01-01,US
21,70,+70%,/trends/explore?q=/m/07gyp7&date=2021-01-01+20...,/m/07gyp7,McDonald's,Fast food chain,2021-01-01,US


In [327]:
#add column teaser to the df cleaned dataframe
df=df.merge(df_subset['teaser'], left_on=df_subset['id'], right_on=df['id'])
df=df.drop(['key_0'], axis=1)
df=df.drop(['keywordStrings'], axis=1)
#add column with full text of the article into the dataframe
# df=df.merge(df_subset['text'], left_on=df['id'], right_on=df_subset['id'] )

In [329]:
#remove all NaN fields
df_clean=df.dropna(how='any')
df_clean_copy = df_clean.copy(deep=True)
##TODO
#df_clean['cleanFocusCategory']=='Law and Justice' replace with Law-and-Justice  and other categories

df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Law and Justice'], 'Law-and-Justice')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Nature and Environment'], 'Nature-and-Environment')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Human Rights'], 'Human-Rights')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Cars and Transportation'], 'Cars-and-Transportation')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Press Freedom'], 'Press-Freedom')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Digital World'], 'Digital-World')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Rule of Law'], 'Rule-of-Law')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Learning German'], 'Learning-German')
df_clean['cleanFocusCategory'] = df_clean['cleanFocusCategory'].replace(['Freedom of Speech'], 'Freedom-of-Speech')

/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/anya_m/Documents/git_dw/venv/lib64/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [330]:
df_clean['cleanFocusCategory'].unique()

array(['History', 'Politics', 'Society', 'Culture', 'Media',
       'Law-and-Justice', 'Catastrophe', 'Lifestyle', 'Business',
       'Travel', 'Sports', 'Science', 'Health', 'Nature-and-Environment',
       'Cars-and-Transportation', 'Education', 'Religion', 'Human-Rights',
       'Technology', 'Learning-German', 'Digital-World', 'Migration',
       'Offbeat', 'Innovation'], dtype=object)

In [337]:
df_clean_single_kw=df_clean.explode('keywordStringsCleanAfterFuzz')

label=df_clean['cleanFocusCategory'].apply(lambda x: ('__label__' +x+' '))
kw=df_clean['keywordStringsCleanAfterFuzz'].apply(lambda x: (', '.join(x)))
teaser=df_clean['teaser']


In [340]:
df_clean_single_kw=df_clean_single_kw.dropna(how='any')

In [345]:
single_kw_label=df_clean_single_kw['cleanFocusCategory'].apply(lambda x: ('__label__' +x+' '))
single_kw=df_clean_single_kw['keywordStringsCleanAfterFuzz']
print(single_kw_label+single_kw)

In [ ]:
print(label+teaser)

In [316]:
print(label+kw)

0        __label__History Margaret Thatcher, Helmut Koh...
1        __label__Politics Syria, Russia, rebels, cease...
2        __label__Politics Syria, Yemen, war, Bashar al...
3        __label__Politics Syria, Russia, Donald Trump,...
4                          __label__Politics Angela Merkel
                               ...                        
69427    __label__Politics Putin, New Year, Coronavirus...
69428    __label__Politics Germany, Olaf Scholz, New Ye...
69429    __label__Catastrophe Colorado, wildfires, Boul...
69430    __label__Business RCEP, Regional Comprehensive...
69431    __label__Health France, birth control, contrac...
Length: 68481, dtype: object


In [396]:
label_kw=label+kw
label_single_kw=single_kw_label+single_kw
label_teaser=label+teaser

In [397]:
# split dataset into train and test subsets
import numpy as np
from sklearn.model_selection import train_test_split as tts

def split_dataset(dataset, label):
    train_set, test_set=tts(dataset, random_state=0, test_size=0.33, stratify=label)
    return train_set, test_set

In [403]:
label_kw_train,label_kw_test=split_dataset(label_kw,label)
label_single_kw_train,label_single_kw_test=split_dataset(label_single_kw,single_kw_label)
label_teaser_train,label_teaser_test=split_dataset(label_teaser,label)

In [348]:
# save training/validation file for fasttext
#label+single keyword (all keywords using pd.explode)
from tqdm import tqdm
!rm ./fasttext/single_kw.txt
with open('./fasttext/single_kw.txt', 'a') as f:
    for i, el in tqdm(enumerate(single_kw_label)):
        #print(i)
        #print(kw[i])
        print(el+single_kw.iloc[i],file=f)

rm: cannot remove ‘./fasttext/single_kw.txt’: No such file or directory


358724it [00:06, 52690.92it/s]


In [349]:
# save training/validation file for fasttext
#label+teaser
from tqdm import tqdm
!rm ./fasttext/teaser.txt
with open('./fasttext/teaser.txt', 'a') as f:
    for i, el in tqdm(enumerate(label)):
        #print(i)
        #print(kw[i])
        print(el+teaser.iloc[i],file=f)

68481it [00:01, 49098.77it/s]


In [350]:
# save training/validation file for fasttext
#label+all DW keywords
from tqdm import tqdm
!rm ./fasttext/output.txt
with open('./fasttext/output.txt', 'a') as f:
    for i, el in tqdm(enumerate(label)):
        #print(i)
        #print(kw[i])
        print(el+kw.iloc[i],file=f)

68481it [00:01, 55123.25it/s]


In [482]:
import os
def save_txt(out_name,data_to_save):
    from tqdm import tqdm
    #!rm $out_name
    if os.path.isfile(out_name):
        os.remove(out_name)
    with open(out_name,'a') as f:
        for i,el in tqdm(enumerate(data_to_save)):
            print(el,file=f)
    input_file=out_name
    return input_file

In [515]:
all_kw_train=save_txt('./fasttext/all_kw_train.txt',label_kw_train)
all_kw_test=save_txt('./fasttext/all_kw_test.txt',label_kw_test)
#all_kw_train
#!head ./fasttext/all_kw_train.txt

45882it [00:00, 385445.01it/s]
22599it [00:00, 350453.20it/s]


In [536]:
single_kw_train=save_txt('./fasttext/single_kw_train.txt',label_single_kw_train)
single_kw_test=save_txt('./fasttext/single_kw_test.txt',label_single_kw_test)

!cat ./fasttext/single_kw_train.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/single_kw_train.preprocessed.txt
!cat ./fasttext/single_kw_test.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/single_kw_test.preprocessed.txt

single_kw_train_prec='./fasttext/single_kw_train.preprocessed.txt'
single_kw_test_prec='./fasttext/single_kw_test.preprocessed.txt'


240345it [00:00, 282301.57it/s]
118379it [00:00, 262190.10it/s]


In [545]:
teaser_train=save_txt('./fasttext/teaser_train.txt',label_teaser_train)
teaser_test=save_txt('./fasttext/teaser_test.txt',label_teaser_test)

!cat ./fasttext/teaser_train.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/teaser_train.preprocessed.txt
!cat ./fasttext/teaser_test.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/teaser_test.preprocessed.txt

teaser_train_prec='./fasttext/teaser_train.preprocessed.txt'
teaser_test_prec='./fasttext/teaser_test.preprocessed.txt'


45882it [00:00, 121746.47it/s]
22599it [00:00, 118641.79it/s]


### Trying Fasttext
- DW all kw + DW cats  - training + validation
- DW all kw for training, DW single kw for validation
- DW single kw for training + validation
- DW teaser + DW cats          - training + validation

implement train_test_split for splitting dataset

- DW teaser / kw for training , google for ???
- - validation based on similarity of google to DW kw (rapidfuzz? cdist / word embeddings using fasstext)

- - make graphics to compare categories from DW and Google

- DW text for training, DW kw for validation
- - clean html (beautiful soup?)

In [351]:
import fasttext

In [247]:
#splitiing dataset into training and validation sets

!head -n 50000 ./fasttext/output.txt > ./fasttext/output.train
!tail -n 20000 ./fasttext/output.txt > ./fasttext/output.valid

In [352]:
#splitiing dataset into training and validation sets
!cat ./fasttext/output.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/output.preprocessed.txt

!head -n 50000 ./fasttext/output.preprocessed.txt > ./fasttext/output_cl.train
!tail -n 20000 ./fasttext/output.preprocessed.txt > ./fasttext/output_cl.valid

In [365]:
# single kw 
#preprocessing and 
#splittig into training and validation sets
!cat ./fasttext/single_kw.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > ./fasttext/single_kw.preprocessed.txt

!head -n 300000 ./fasttext/single_kw.preprocessed.txt > ./fasttext/single_kw_cl.train
!tail -n 60000 ./fasttext/single_kw.preprocessed.txt > ./fasttext/single_kw_cl.valid

In [551]:
def fasttext_classifier(train_file,test_file,lr=1.0,epoch=12,k=1,wordNgrams=1):
    model = fasttext.train_supervised(input=train_file, lr=lr, epoch=epoch)
    #model.test(test_file,k=k)
    print(model.test(test_file,k=1))
    print(model.test(test_file,k=2))
    print(model.test(test_file,k=3))
    print(model.test(test_file,k=4))
    print(model.test(test_file,k=5))
    return model

In [539]:
model_all_kw_check=fasttext_classifier(all_kw_train,all_kw_test,lr=0.7)
#model_all_kw_check.test(all_kw_test)

Read 0M words
Number of words:  36772
Number of labels: 24
Progress: 100.0% words/sec/thread:  448071 lr:  0.000000 avg.loss:  0.536056 ETA:   0h 0m 0s


(22599, 0.6917120226558697, 0.6917120226558697)
(22599, 0.41285012611177485, 0.8257002522235497)
(22599, 0.29516055282682124, 0.8854816584804638)
(22599, 0.22923580689411036, 0.9169432275764414)
(22599, 0.18739767246338335, 0.9369883623169166)


(22599, 0.6917120226558697, 0.6917120226558697)

In [552]:
model_single_kw_check=fasttext_classifier(single_kw_train,single_kw_test,lr=0.7,wordNgrams=1)
#model_single_kw_check.test(single_kw_test)

Read 0M words
Number of words:  28776
Number of labels: 24
Progress: 100.0% words/sec/thread:  195677 lr:  0.000000 avg.loss: 10.151832 ETA:   0h 0m 0s


(118379, 0.5124050718455131, 0.5124050718455131)
(118379, 0.25624054942177243, 0.5124810988435449)
(118379, 0.17112832512523335, 0.5133849753757)
(118379, 0.1311360122994788, 0.5245440491979152)
(118379, 0.12260451600368309, 0.6130225800184155)


In [553]:
model_single_kw_check=fasttext_classifier(single_kw_train,single_kw_test,lr=0.7,wordNgrams=5)
#model_single_kw_check.test(single_kw_test)

Read 0M words
Number of words:  28776
Number of labels: 24
Progress: 100.0% words/sec/thread:  244412 lr:  0.000000 avg.loss: 10.170701 ETA:   0h 0m 0s


(118379, 0.5066354674393262, 0.5066354674393262)
(118379, 0.2533515234965661, 0.5067030469931322)
(118379, 0.16922765017443972, 0.5076829505233191)
(118379, 0.12984144147188267, 0.5193657658875307)
(118379, 0.12065315638753495, 0.6032657819376748)


In [541]:
model_single_kw_check_prec=fasttext_classifier(single_kw_train_prec,single_kw_test_prec,lr=0.7)
#model_single_kw_check_prec.test(single_kw_test_prec)

Read 0M words
Number of words:  28595
Number of labels: 24
Progress: 100.0% words/sec/thread:  223871 lr:  0.000000 avg.loss: 10.209421 ETA:   0h 0m 0s


(118379, 0.5109183216617812, 0.5109183216617812)
(118379, 0.25548872688568075, 0.5109774537713615)
(118379, 0.17073411106136505, 0.5122023331840951)
(118379, 0.13140844237575922, 0.5256337695030369)
(118379, 0.12208584292822207, 0.6104292146411103)


(118379, 0.5109183216617812, 0.5109183216617812)

In [548]:
model_teaser=fasttext_classifier(teaser_train,teaser_test)

Read 1M words
Number of words:  89320
Number of labels: 24
Progress:  98.9% words/sec/thread:  820660 lr:  0.011198 avg.loss:  0.601301 ETA:   0h 0m 0s

(22599, 0.6690119031815567, 0.6690119031815567)
(22599, 0.40249568564980753, 0.8049913712996151)
(22599, 0.2884198415859109, 0.8652595247577326)


Progress: 100.0% words/sec/thread:  819037 lr:  0.000000 avg.loss:  0.594989 ETA:   0h 0m 0s


(22599, 0.22408071153590867, 0.8963228461436347)
(22599, 0.18339749546440107, 0.9169874773220054)


In [549]:
model_teaser=fasttext_classifier(teaser_train_prec,teaser_test_prec)

Read 1M words
Number of words:  54981
Number of labels: 24
Progress: 100.0% words/sec/thread:  794113 lr:  0.000000 avg.loss:  0.772094 ETA:   0h 0m 0s100.0% words/sec/thread:  794117 lr: -0.000005 avg.loss:  0.772094 ETA:   0h 0m 0s


(22599, 0.6777291030576574, 0.6777291030576574)
(22599, 0.4076286561352272, 0.8152573122704544)
(22599, 0.29272681682080326, 0.8781804504624099)
(22599, 0.22819593787335724, 0.9127837514934289)
(22599, 0.18667197663613433, 0.9333598831806718)


In [550]:
model_teaser_single=fasttext_classifier(teaser_train_prec,single_kw_test_prec)

Read 1M words
Number of words:  54981
Number of labels: 24
Progress: 100.0% words/sec/thread:  999731 lr:  0.000000 avg.loss:  0.819998 ETA:   0h 0m 0s


(118379, 0.3832774394107063, 0.3832774394107063)
(118379, 0.26068812880662956, 0.5213762576132591)
(118379, 0.20177565277625253, 0.6053269583287576)
(118379, 0.16474416915162318, 0.6589766766064927)
(118379, 0.13969876413890978, 0.6984938206945489)


In [373]:
model_best_all_single = fasttext.train_supervised(input="./fasttext/output.preprocessed.txt", autotuneValidationFile="./fasttext/single_kw.preprocessed.txt")
model_best_all_single.test("./fasttext/single_kw.preprocessed.txt")

Progress: 100.0% Trials:   13 Best score:  0.526998 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  34650
Number of labels: 24
Progress: 100.0% words/sec/thread:  535450 lr:  0.000000 avg.loss:  1.001028 ETA:   0h 0m 0s


(358724, 0.522613485576655, 0.522613485576655)

In [374]:
#automatic search for best hyperparameters https://fasttext.cc/docs/en/autotune.html 
#last 5 minutes
train_file="./fasttext/single_kw_cl.train"
valid_file='./fasttext/single_kw_cl.valid'
model_best_single = fasttext.train_supervised(input=train_file, autotuneValidationFile=valid_file)
model_best_single.test(valid_file)

Progress: 100.0% Trials:   10 Best score:  0.527900 ETA:   0h 0m 0s
Training again with best arguments
Read 1M words
Number of words:  31928
Number of labels: 24
Progress: 100.0% words/sec/thread:  227185 lr:  0.000000 avg.loss:  1.242923 ETA:   0h 0m 0s


(60000, 0.518, 0.518)

In [303]:
#automatic search for best hyperparameters https://fasttext.cc/docs/en/autotune.html 
#last 5 minutes
model_best = fasttext.train_supervised(input=train_file, autotuneValidationFile='./fasttext/output_cl.valid')
model_best.test('./fasttext/output_cl.valid')

Progress: 100.0% Trials:   23 Best score:  0.658736 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  31769
Number of labels: 22
Progress: 100.0% words/sec/thread:  741156 lr:  0.000000 avg.loss:  0.837053 ETA:   0h 0m 0s


In [ ]:
#The output are the number of samples (here 14999), 
# the precision at one (0.6968)  69% and 
# the recall at one (0.0541).

#The precision is the number of correct labels among the labels predicted by fastText. 
# The recall is the number of labels that successfully were predicted,
# among all the real labels. 

With a few steps, we were able to go from a precision at one of 12.4% to 59.9%. Important steps included:

- preprocessing the data ;
- changing the number of epochs (using the option -epoch, standard range [5 - 50]) ;
- changing the learning rate (using the option -lr, standard range [0.1 - 1.0]) ;
- using word n-grams (using the option -wordNgrams, standard range [1 - 5]).

In [284]:
# classifier with cleaned teaser tested on teaser
train_file="./fasttext/teaser_cl.train"
model_file="./fasttext/model_teaser_cl_output.bin"
test_file="./fasttext/teaser_cl.valid"

model_teaser_cl = fasttext.train_supervised(input=train_file, lr=0.9, epoch=25)
#saving the model
model_teaser_cl.save_model(model_file)
#test model

model_teaser_cl.test(test_file)

Read 1M words
Number of words:  51697
Number of labels: 21
Progress: 100.0% words/sec/thread: 1067246 lr:  0.000000 avg.loss:  0.376231 ETA:   0h 0m 0s


(14794, 0.5892929566040287, 0.5892929566040287)

In [288]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, epoch=12)
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

Read 2M words
Number of words:  66706
Number of labels: 24
Progress: 100.0% words/sec/thread: 1065250 lr:  0.000000 avg.loss:  0.680163 ETA:   0h 0m 0s


(68481, 0.6716899577985135, 0.6716899577985135)

In [ ]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, epoch=50, bucket=200000, dim=50, loss='hs')
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

In [289]:
# classifier with cleaned teaser tested on DW kw
train_file="./fasttext/teaser.preprocessed.txt" #all cleaned teasers +cat
model_file="./fasttext/model_teaser_pred_kw.bin"
test_file="./fasttext/output.preprocessed.txt" #all cleaned dw kw + cat

model_teaser_kw = fasttext.train_supervised(input=train_file, lr=1.0, wordNgrams=5, epoch=50, bucket=200000, dim=50, loss='hs')
#saving the model
model_teaser_kw.save_model(model_file)
#test model

model_teaser_kw.test(test_file)

Read 2M words
Number of words:  66706
Number of labels: 24
Progress: 100.0% words/sec/thread:  582548 lr:  0.000000 avg.loss:  5.385403 ETA:   0h 0m 0s


(68481, 0.625472758867423, 0.625472758867423)

In [ ]:
#google keywords to dw categories


In [ ]:
# # Skipgram model :
# model = fasttext.train_unsupervised('data.txt', model='skipgram')

# # or, cbow model :
# model = fasttext.train_unsupervised('data.txt', model='cbow')


# print(model.words)   # list of words in dictionary
# print(model['king']) # get the vector of the word 'king'